In [1]:
ids = {}
ids["ANALOG_ID"] = 1000
ids["ONOFF_ID"] = 1001
ids["TEMPERATURE_DHT11_ID"] = 1002
ids["HUMIDITY_DHT11_ID"] = 1003
ids["DIGITAL_ALERT_ID"] = 1004
ids["INIT_SERVICE_ID"] = 1005
ids["TEMPERATURE_DHT22_ID"] = 1006
ids["HUMIDITY_DHT22_ID"] = 1007
ids["PRESSURE_ID"] = 1008



In [64]:
class Service:
    def __init__(self, serviceId, idKey, propertyName, setup):
        self.serviceId = serviceId
        self.idKey = idKey
        self.serviceKey = idKey[:-3]
        self.propertyName = propertyName
        self.setup = setup

In [88]:
analog = Service(1000, "ANALOG_ID", "analog", "INPUT")
onoff = Service(1001, "ONOFF_ID", "onoff", "OUTPUT")
temperature11 = Service(1002, "TEMPERATURE_DHT11_ID", "temperature11", "INPUT")
humidity11 = Service(1003, "HUMIDITY_DHT11_ID", "humidity11", "INPUT")
alert = Service(1004, "DIGITAL_ALERT_ID", "alert", "INPUT")
init = Service(1005, "INIT_SERVICE_ID", "init", "INPUT")
temperature22 = Service(1006, "TEMPERATURE_DHT22_ID", "temperature22", "INPUT")
humidity22 = Service(1007, "HUMIDITY_DHT22_ID", "humidity22", "INPUT")
pressure = Service(1008, "PRESSURE_ID", "pressure", "INPUT")

In [89]:
deviceName = "arduinonano"
rx, tx = "2", "3"
setup = [
    (analog, "A0"),
    (analog, "A1"),
    (analog, "A2"),
    (analog, "A3"),
    (analog, "A4"),
    (analog, "A5"),
    (alert, "4"),
    (alert, "5"),
    (alert, "6"),
    (alert, "7"),
    (temperature22, "8"),
    (humidity22, "8"),
    (init, "9"),
    (onoff, "10"),
    (onoff, "11"),
    (onoff, "12")
]

In [90]:
setup[list(map(lambda x: x[0], setup)).index(init)][1]

'9'

In [91]:
paste = """
#include "user_properties.h"
#include "services.h"

#define DEVICE_NAME "%s"
#define RASPBERRY_PORT 8080
#define UDP_PORT       59743
#define ARDUINO_PORT   8080

// used services
""" % (deviceName)

for i in set(map(lambda x: x[0], setup)):
    paste += "#define " + i.serviceKey + "\n"

    
# paste += "\n// pins for services\n"
# count = 0
# for i in setup:
#     paste += "#define " + i[0].propertyName + "_" + str(count) + " " + i[1] + "\n"
#     count += 1
    
paste += "\n// pins for communication with esp8266\n"
paste += "#define RX " + rx + " // attach to TX of esp\n"
paste += "#define TX " + tx + " // attach to RX of esp\n"

paste += "\n// static services array, pay attention to the defined pins\n"
paste += "int SERVICES[] = {%s};\n" % ", ".join(map(lambda x: x[0].idKey, setup))
paste += "int PINS[] = {%s};\n" % ", ".join(map(lambda x: x[1], setup))
paste += "#define SERVICES_STR \"%s\" \n" % ";".join(map(lambda x: str(x[0].serviceId), setup)) 
paste += "#define SERVICES_COUNT %s\n" % len(setup) 
paste += "#define DIGITAL_ALERT_COUNT %s\n" % list(map(lambda x: x[0], setup)).count(alert)
if (init in list(map(lambda x: x[0], setup))):
    paste += "#define INIT_PIN " + setup[list(map(lambda x: x[0], setup)).index(init)][1] + "\n"
paste += "// will be called on arduino 'setup()'\n"
paste += "void setupConfiguration() {\n"

for i in setup:
    paste += "\tpinMode(%s, %s);\n" % (i[1], i[0].setup)
paste += "}\n"

print(paste)


#include "user_properties.h"
#include "services.h"

#define DEVICE_NAME "arduinonano"
#define RASPBERRY_PORT 8080
#define UDP_PORT       59743
#define ARDUINO_PORT   8080

// used services
#define INIT_SERVICE
#define TEMPERATURE_DHT22
#define DIGITAL_ALERT
#define HUMIDITY_DHT22
#define ONOFF
#define ANALOG

// pins for communication with esp8266
#define RX 2 // attach to TX of esp
#define TX 3 // attach to RX of esp

// static services array, pay attention to the defined pins
int SERVICES[] = {ANALOG_ID, ANALOG_ID, ANALOG_ID, ANALOG_ID, ANALOG_ID, ANALOG_ID, DIGITAL_ALERT_ID, DIGITAL_ALERT_ID, DIGITAL_ALERT_ID, DIGITAL_ALERT_ID, TEMPERATURE_DHT22_ID, HUMIDITY_DHT22_ID, INIT_SERVICE_ID, ONOFF_ID, ONOFF_ID, ONOFF_ID};
int PINS[] = {A0, A1, A2, A3, A4, A5, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12};
#define SERVICES_STR "1000;1000;1000;1000;1000;1000;1004;1004;1004;1004;1006;1007;1005;1001;1001;1001" 
#define SERVICES_COUNT 16
#define DIGITAL_ALERT_COUNT 4
#define INIT_PIN 9
// will be called on